In [15]:
import os
import cPickle as pickle
import gzip

In [23]:
import matplotlib.pyplot as plt
%pylab inline
import numpy as np

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [50]:
from lasagne.layers import DenseLayer
from lasagne.layers import InputLayer
from lasagne.layers import DropoutLayer
from lasagne.layers import Conv2DLayer
from lasagne.layers import MaxPool2DLayer
from lasagne.layers import get_all_layers, get_output
from lasagne.nonlinearities import softmax
from lasagne.updates import adam
from lasagne.layers import get_all_params

In [25]:
from nolearn.lasagne import NeuralNet
from nolearn.lasagne import TrainSplit
from nolearn.lasagne import objective

----

In [70]:
with gzip.open("../../data/mnist.pkl.gz") as f:
    train_data, _, _ = pickle.load(f)
    X, y = train_data

In [72]:
X, y = X[0:10], y[0:10]
X = X.reshape((X.shape[0], 1, 28, 28)).astype("float32")
y = y.astype("int32")

----

In [60]:
layers0 = [
    (InputLayer, {'shape': (None, X.shape[1], X.shape[2], X.shape[3])}),
    (Conv2DLayer, {'num_filters': 96, 'filter_size': 5}),
    (Conv2DLayer, {'num_filters': 96, 'filter_size': 3}),
    (Conv2DLayer, {'num_filters': 96, 'filter_size': 3}),
    (Conv2DLayer, {'num_filters': 96, 'filter_size': 3}),
    (Conv2DLayer, {'num_filters': 96, 'filter_size': 3}),
    (MaxPool2DLayer, {'pool_size': 2}),
    (Conv2DLayer, {'num_filters': 128, 'filter_size': 3}),
    (Conv2DLayer, {'num_filters': 128, 'filter_size': 3}),
    (Conv2DLayer, {'num_filters': 128, 'filter_size': 3}),
    (MaxPool2DLayer, {'pool_size': 2}),
    (DenseLayer, {'num_units': 64}),
    (DropoutLayer, {}),
    (DenseLayer, {'num_units': 10, 'nonlinearity': softmax}),
]

In [61]:
def dump_intermediate_layer(layer, test_input):
    input_var = get_all_layers(layer)[0].input_var
    layer_out = get_output(layer, input_var)
    f = theano.function([input_var], layer_out)
    def after_epoch(net, info):
        print f(test_input)
    return after_epoch

In [74]:
net0.update_learning_rate

0.0002

In [67]:
net0 = NeuralNet(
    layers=layers0,
    max_epochs=10,
    update=adam,
    update_learning_rate=0.0002,
    objective_l2=0.0025,
    train_split=TrainSplit(eval_size=0.0),
    verbose=1,
)
net0.initialize()
net0.on_epoch_finished = [
    dump_intermediate_layer( net0.get_all_layers()[-2], X[0:1])    
]

In [68]:
for layer in net0.get_all_layers():
    print layer, layer.output_shape

<lasagne.layers.input.InputLayer object at 0x116e49350> (None, 1, 28, 28)
<lasagne.layers.conv.Conv2DLayer object at 0x116e49c90> (None, 96, 24, 24)
<lasagne.layers.conv.Conv2DLayer object at 0x110a389d0> (None, 96, 22, 22)
<lasagne.layers.conv.Conv2DLayer object at 0x10e332fd0> (None, 96, 20, 20)
<lasagne.layers.conv.Conv2DLayer object at 0x11872a4d0> (None, 96, 18, 18)
<lasagne.layers.conv.Conv2DLayer object at 0x118814290> (None, 96, 16, 16)
<lasagne.layers.pool.MaxPool2DLayer object at 0x11882ee90> (None, 96, 8, 8)
<lasagne.layers.conv.Conv2DLayer object at 0x11883ded0> (None, 128, 6, 6)
<lasagne.layers.conv.Conv2DLayer object at 0x118746cd0> (None, 128, 4, 4)
<lasagne.layers.conv.Conv2DLayer object at 0x11876e810> (None, 128, 2, 2)
<lasagne.layers.pool.MaxPool2DLayer object at 0x116a23390> (None, 128, 1, 1)
<lasagne.layers.dense.DenseLayer object at 0x116a23f50> (None, 64)
<lasagne.layers.noise.DropoutLayer object at 0x119e3d350> (None, 64)
<lasagne.layers.dense.DenseLayer object 

In [73]:
net0.fit(X[0:1],y[0:1])

# Neural Network with 749450 learnable parameters

## Layer information

  #  name         size
---  -----------  --------
  0  input0       1x28x28
  1  conv2d1      96x24x24
  2  conv2d2      96x22x22
  3  conv2d3      96x20x20
  4  conv2d4      96x18x18
  5  conv2d5      96x16x16
  6  maxpool2d6   96x8x8
  7  conv2d7      128x6x6
  8  conv2d8      128x4x4
  9  conv2d9      128x2x2
 10  maxpool2d10  128x1x1
 11  dense11      64
 12  dropout12    64
 13  dense13      10

[[ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.01763937  0.          0.          0.02200288
   0.          0.          0.02756306  0.          0.          0.
   0.01850183  0.05389618  0.          0.          0.01612351  0.          0.
   0.          0.          0.          0.          0.02602817  0.04100121
   0.          0.0208814   0.01848322  0.03867007  0.          0.          0.
   0.05974561  0.00022636  0.          0.          0.          0.          

NeuralNet(X_tensor_type=None,
     batch_iterator_test=<nolearn.lasagne.base.BatchIterator object at 0x10d69f0d0>,
     batch_iterator_train=<nolearn.lasagne.base.BatchIterator object at 0x10d693f10>,
     check_input=True, custom_scores=None,
     layers=[(<class 'lasagne.layers.input.InputLayer'>, {'shape': (None, 1, 28, 28)}), (<class 'lasagne.layers.conv.Conv2DLayer'>, {'filter_size': 5, 'num_filters': 96}), (<class 'lasagne.layers.conv.Conv2DLayer'>, {'filter_size': 3, 'num_filters': 96}), (<class 'lasagne.layers.conv.Conv2DLayer'>, {'fil...ne.layers.dense.DenseLayer'>, {'num_units': 10, 'nonlinearity': <function softmax at 0x10c695c80>})],
     loss=None, max_epochs=10, more_params={},
     objective=<function objective at 0x10d69cd70>, objective_l2=0.0025,
     objective_loss_function=<function categorical_crossentropy at 0x10d141f50>,
     on_batch_finished=[],
     on_epoch_finished=[<function after_epoch at 0x1189c8ed8>],
     on_training_finished=[],
     on_training_started